# Kangaroo Dashboard

#### Links

``https://medium.com/plotly/how-to-create-a-beautiful-interactive-dashboard-layout-in-python-with-plotly-dash-a45c57bb2f3c``\
``https://fonts.google.com/?query=poppins&preview.text=poppins.``\
``https://github.com/plotly/datasets/tree/master/dash-layout-tutorial``\
``https://plotly.com/python/reference/layout/``

## Libraries

In [1]:
from dash import Dash, html, dcc, dash_table
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output, State

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import random

## Data

In [2]:
df = pd.read_csv('./kangaroo.csv', encoding='utf-8')  

In [3]:
year_list = np.sort(df.Year.unique())
year_list = year_list.astype('str') 
year_list = np.insert(year_list, 0, 'All')

In [18]:
colors_bright =  ['#ff3d44', '#74eafc', '#80f569', '#ffb85c', '#e969f5', '#fcfa62']
#                  red        blue       green      orange     violet     yellow
colors_rainbow = ['red',     'orange',  'yellow',  'green',   'blue',    'purple']
#                  red        orange     yellow     green      blue       violet
colors_pastel =  ['#F2C6DE', '#C6DEF1', '#C9E4DE', '#F7D9C4', '#DBCDF0', '#FAEDCB']
#                  red        blue       green      orange     violet     yellow
colors = colors_bright

## App

In [5]:
app = Dash(__name__, external_stylesheets=[dbc.themes.FLATLY])

## Charts

### species population by year

In [6]:
def species_by_year(color_choice, start_year, end_year):

    if color_choice == 1:
        colors = colors_bright
    if color_choice == 2:
        colors = colors_rainbow
    if color_choice == 3:
        colors = colors_pastel
    
    df_a = df.groupby(['Year']).sum()
    df_a.drop('State', axis=1, inplace=True)
    df_a.reset_index(inplace=True)
    df_a = df_a[(start_year-2001) : (end_year-2001)+1]
    fig_a = go.Figure()
    for i in range(len(df_a.columns[1:])):
        fig_a.add_trace(go.Scatter(x = df_a.Year.to_list(), 
                                   y = df_a[df_a.columns[1:][i]].to_list(), 
                                   hovertemplate='%{y:.0f}'+'<br>%{x}',
                                   opacity = 0.8, 
                                   mode = 'lines', 
                                   name = df_a.columns[1:][i], 
                                   marker = dict(color=colors[i]),
                                   line = dict(width=6.0),))
    fig_a.update_layout(title={'text': "Species Population",
                               'x':0.45, 'y':0.95,
                               'xanchor': 'center'},
                        #xaxis_title = 'Years', 
                        #yaxis_title = 'Population', 
                        title_font_size= 24,
                        template = 'plotly_dark', 
                        width = 790, height = 730)
    return dcc.Graph(figure=fig_a)

### province population by year

In [7]:
def province_by_year(color_choice, start_year, end_year):

    if color_choice == 1:
        colors = colors_bright
    if color_choice == 2:
        colors = colors_rainbow
    if color_choice == 3:
        colors = colors_pastel
    
    df_b = df.loc[:]
    df_b['Total'] = df_b.sum(axis=1, numeric_only=True)
    df_b = df_b[['Year','State','Total']]
    df_b = df_b.groupby(['Year', 'State']).mean()
    df_b = df_b.pivot_table(values='Total', index='Year', columns='State').reset_index()
    df_b.rename_axis(None, axis=1, inplace=True)
    df_b = df_b[(start_year-2001) : (end_year-2001)+1]
    fig_b = go.Figure()
    for i in range(len(df_b.columns[1:])):
        fig_b.add_trace(go.Scatter(x = df_b.Year.to_list(), 
                                   y = df_b[df_b.columns[1:][i]].to_list(), 
                                   hovertemplate='%{y:.0f}'+'<br>%{x}',
                                   opacity = 0.8,
                                   mode = 'lines', 
                                   marker = dict(color=colors[2+i]), 
                                   name = df_b.columns[1:][i],
                                   line = dict(width=6.0),))
    fig_b.update_layout(title=dict(font=dict(size=24))) 
    fig_b.update_layout(title={'text': "Provincial Population",
                                'x':0.45, 'y':0.95,
                                'xanchor': 'center'},
                        #xaxis_title = 'Years', 
                        #yaxis_title='Population', 
                        title_font_size= 24,
                        template = 'plotly_dark', 
                        width=790, height=730)
    return dcc.Graph(figure=fig_b)

### species histogram 

In [8]:
def species_histogram():
    df_h2 = df.rename(columns={'State':'Province'})
    df_h2 = df_h2.groupby(['Province']).mean()
    df_h2.drop('Year', axis=1, inplace=True)
    df_h2 = df_h2.T
    df_h2.reset_index(inplace=True)
    df_h2.rename(columns={'index':'Species'}, inplace=True)
    
    fig_c = px.histogram(data_frame = df_h2, x = 'Species', y = df_h2.columns, barmode='group')
    fig_c.update_layout(title=dict(font=dict(size=24))) 
    fig_c.update_layout(title={'text': "Provincial Population by Species",
                                'x':0.45, 'y':0.95,
                                'xanchor': 'center'},
                        xaxis_title = '', yaxis_title='', 
                        title_font_size= 24,
                        template = 'plotly_dark', 
                        width=790, height=730)
    return dcc.Graph(figure=fig_c)

### scatter stars

In [9]:
def scatter_stars():
    star_fig = go.Figure(
        go.Scattergl(
            x = np.random.randn(1000),
            y = np.random.randn(1000),
            mode='markers',
            marker=dict(color=random.sample(['#ecf0f1']*500 + ["#3498db"]*500, 1000), line_width=1)
        )
    )
    star_fig.update_layout(template = 'plotly_dark', width=790, height=730,
                      xaxis_visible=False, yaxis_visible=False, 
                      showlegend=False, margin=dict(l=0,r=0,t=0,b=0))
    
    return dcc.Graph(figure=star_fig)

## Table(s)

### All Data

In [10]:
def table_data():
    return dash_table.DataTable(
                data=df.to_dict('records'),
                columns=[{'id': c, 'name': c} for c in df.columns],
                page_size = 22,
                style_cell={'text-align': 'center', 'font_family': 'Oswald'},
                style_header={
                    'backgroundColor': '#d7d7d7',
                    'color': 'black',
                    'fontWeight': 'bold'
                },
                style_data={
                    'color': 'black',
                    'backgroundColor': '#708090'
                },
                style_data_conditional=[
                    {
                        'if': {'row_index': 'odd'},
                        'backgroundColor': '#d3d3d3',
                    }
                ],
                style_table={'padding-left': 100, 'fontSize':20, 'font-family':'Eurostile'},
            )  

## Callbacks

### Radio

#### Display Type

In [11]:
@app.callback([Output(component_id='radio-out', component_property='children')],
              [Input(component_id='select-radio', component_property='value')],
              [State("select-radio","options")])

def display_radio_choice(value_radio, options_radio):
    label_radio = [opt['label'] for opt in options_radio if opt['value'] == value_radio][0]
    
    return [html.Div(html.H3(label_radio), className='Output'),]        

#### Hide Other Inputs

In [12]:
@app.callback( [Output(component_id='dropdown-one', component_property='style'),
                Output(component_id='dropdown-two', component_property='style'),
                Output(component_id='range-slider', component_property='style'),
                Output(component_id='spacer', component_property='style')],
                Input(component_id='select-radio',component_property='value'))

def disable_dropdowns(radio_selection):
    if radio_selection == 1: # Graph
        return {'display': ''},     {'display': ''},     {'margin-top': 35},   {'height': 0}
    else: 
        return {'display': 'none'}, {'display': 'none'}, {'display': 'none'},  {'height': 195}

### Dropdown and Slider Output

In [13]:
@app.callback([Output(component_id='option-one', component_property='children'),
               Output(component_id='option-two', component_property='children'),
               Output(component_id='option-three', component_property='children')],
              [Input(component_id='select-one', component_property='value'),
               Input(component_id='select-two', component_property='value'),
               Input(component_id='select-range', component_property='value')],
              [State("select-one","options"),
               State("select-two","options")])

def display_option (value_one, value_two, year_range, options_one, options_two):
    
    label_one =   [opt['label'] for opt in options_one if opt['value'] == value_one][0]
    label_two =   [opt['label'] for opt in options_two if opt['value'] == value_two][0]
    label_three = year_range[0],' to ',year_range[1]
    
    return [html.Div(html.H3(label_one), className='Output'),
            html.Div(html.H3(label_two), className='Output'),
            html.Div(html.H3(label_three), className='Output')]

### Type and Categories

In [14]:
@app.callback([Output(component_id='out-container', component_property='children')],
              [Input(component_id='select-one', component_property='value'),
               Input(component_id='select-two', component_property='value'),
               Input(component_id='select-range', component_property='value'),
               Input(component_id='select-radio', component_property='value')])

def display_star (select_one_value, select_two_value, year_range, select_radio_value):

    if select_radio_value == 1:
        if select_one_value == 1: 
            component = species_by_year(select_two_value, year_range[0], year_range[1])
        else: 
            component = province_by_year(select_two_value, year_range[0], year_range[1])
    else: # Chart or Table
        if select_radio_value == 2:
            component = species_histogram()
        else:
            component = table_data()

    return [html.Div(component, style={'display': 'flex'})]

## Layout

In [15]:
def null_evens(n):
        return str(n) if n%2==1 else ''
range_markers = dict(zip([int(y) for y in year_list[1:]], 
                         [null_evens(int(y)) for y in year_list[1:]]))

In [16]:
app.layout = dbc.Container([
    html.Div([
        
        html.Div([
            html.H1([
                html.Span("Kangaroo"),
                html.Br(),
                html.Span("Population"),
                html.Br(),
                html.Span("Australia")
            ]),
            html.
            P("Analysis of population of four species of kangaroos by Australian province for the years 2001 to 2011")
        ],  style={"vertical-alignment": "top", "height": 260}),
        
        html.Div([
            html.Div(
                dbc.RadioItems(
                    id = 'select-radio',
                    className='btn-group',
                    inputClassName='btn-check',
                    labelClassName="btn btn-outline-light",
                    labelCheckedClassName="btn btn-light",
                    options=[
                        {"label": "Graph", "value": 1},
                        {"label": "Chart", "value": 2},
                        {"label": "Data", "value": 3}
                    ],
                    value=1,
                    style={'width': '100%'}
                ), style={'width': 309}
            ),
        ], style={'margin-left': 15, 'margin-right': 15, 'display': 'flex'}),
        
        html.Div([
            html.Div([
                html.H2('Category'),
                dcc.Dropdown(
                    id='select-one',
                    options=[
                        {'label': 'Species',  'value': 1},
                        {'label': 'Province', 'value': 2}                   
                    ],
                    value=1,
                    clearable=False,
                    optionHeight=40,
                    className='customDropdown'
                )
            ], id='dropdown-one', style={'height': 60}),
            
            html.Div([
                html.H2('Palette'),
                dcc.Dropdown(
                    id='select-two',
                    options=[
                        {'label': 'Bright', 'value': 1},
                        {'label': 'Rainbow', 'value': 2},
                        {'label': 'Pastel', 'value': 3}
                    ],
                    value=1,
                    clearable=False,
                    optionHeight=40,
                    className='customDropdown'
                )
            ], id='dropdown-two', style={'height': 60}),
            
            html.Div([
                html.H2(''),        
                dcc.RangeSlider(
                    id='select-range',
                    min=2001, max=2011, step=1,
                    marks = range_markers,
                    value=[2001,2011])
            ], id='range-slider', style={'margin-top': 40}),
            
        ], style={'margin-left': 15, 'margin-right': 15, 'margin-top': 30}),

        html.Div(id='spacer'),
        
        html.Div(
            html.Img(src='assets/logo6.png', style={'margin-left': 15, 
                                                    'margin-right': 15, 
                                                    'margin-top': 30, 
                                                    'width': 310})
        )
    ], style={
        'width': 340,
        'margin-left': 35,
        'margin-top': 35,
        'margin-bottom': 35
    }),
    html.Div(
        [         
            html.Div(id='out-container', className='chart-item', style={'width': 790, 'height': 730}),
            
            html.Div([       
                html.H2('Type'),
                html.Div(html.H3("Selected Type"), className='Output', id='radio-out'),
                html.H2('Category'),
                html.Div(html.H3("Selected Plot"), className='Output', id='option-one'),
                html.H2('Color'),
                html.Div(html.H3("Selected Color"), className='Output', id='option-two'),
                html.H2('Years'),
                html.Div(html.H3("Selected Year"), className='Output', id='option-three'),      
            ],  style={'width': 198, 'margin-left':10}),
        ],
        style={
            'width': 990,
            'margin-top': 35,
            'margin-right': 35,
            'margin-bottom': 35,
            'display': 'flex'
        })
],
   fluid=True,
   style={'display': 'flex'},
   className='dashboard-container')

## Launch

In [17]:
if __name__ == '__main__':
    app.run_server(port=8502,
                   debug=True,
                   mode='jupyterlab',
                   dev_tools_ui=True,
                   dev_tools_hot_reload=True, 
                   threaded=True)